In [1]:
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Input, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.optimizers import RMSprop, Adam
from keras import backend as K

In [2]:
height = 299
width = 299
batch_size = 10

train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,
    validation_split=0.1)

train_generator = train_datagen.flow_from_directory(
    'D:\.Downloads\low_res\data',#'/content/drive/My Drive/malariaNN/high_res/data',
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training')
validation_generator = train_datagen.flow_from_directory(
    'D:\.Downloads\low_res\data',#'/content/drive/My Drive/malariaNN/high_res/data',
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation')

epochs = 25
steps = 24804
val_steps = 2754

Found 24804 images belonging to 2 classes.
Found 2754 images belonging to 2 classes.


In [3]:
#inception model
input_shape = None

if K.image_data_format() == 'channels_first':
    input_shape = (3, width, height)
else:
    input_shape = (width, height, 3)

base_model = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=input_shape))

out = base_model.output
out = GlobalAveragePooling2D()(out)
out = Dense(256, activation='relu')(out)
predictions = Dense(1, activation='sigmoid')(out)

model = Model(inputs=base_model.input, outputs=predictions)

LR = 0.0001
RMS = Adam(lr=LR)
model.compile(optimizer=RMS, loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
# own model
input_shape = None

if K.image_data_format() == 'channels_first':
    input_shape = (3, width, height)
else:
    input_shape = (width, height, 3)


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [4]:
history = model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=steps // batch_size,
    validation_data=validation_generator,
    validation_steps=val_steps // batch_size
)

Epoch 1/25
2480/2480 [==============================] - 468s 189ms/step - loss: 0.1318 - accuracy: 0.9574 - val_loss: 0.2572 - val_accuracy: 0.9135
Epoch 2/25
2480/2480 [==============================] - 450s 181ms/step - loss: 0.1073 - accuracy: 0.9652 - val_loss: 0.1725 - val_accuracy: 0.9400
Epoch 3/25
2480/2480 [==============================] - 450s 181ms/step - loss: 0.0963 - accuracy: 0.9675 - val_loss: 0.2117 - val_accuracy: 0.9349
Epoch 4/25
2480/2480 [==============================] - 450s 181ms/step - loss: 0.0918 - accuracy: 0.9700 - val_loss: 0.2768 - val_accuracy: 0.8964
Epoch 5/25
2480/2480 [==============================] - 449s 181ms/step - loss: 0.0839 - accuracy: 0.9713 - val_loss: 0.2348 - val_accuracy: 0.9196
Epoch 6/25
2480/2480 [==============================] - 449s 181ms/step - loss: 0.0769 - accuracy: 0.9746 - val_loss: 0.2034 - val_accuracy: 0.9345
Epoch 7/25
2480/2480 [==============================] - 449s 181ms/step - loss: 0.0712 - accuracy: 0.9766 - val_

In [5]:
accs = 0
for i in range(10):
    accuracy = model.evaluate(
        validation_generator,
        steps=val_steps // batch_size,
    )
    accs += accuracy[1]
print("Avg accuracy: " + str(accs/10))

275/275 [==============================] - 44s 161ms/step - loss: 0.3074 - accuracy: 0.9371
Avg accuracy: 0.9373090922832489


In [14]:
model.save('D:\.Downloads\models\largemodelv2')

INFO:tensorflow:Assets written to: D:\.Downloads\models\largemodelv2\assets


In [7]:
base_model.save_weights('D:\.Downloads\models\largemodelv4\modelweights.h5')